<a href="https://colab.research.google.com/github/FaiazS/Digital-Research-Assistant/blob/main/Deep_Research_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install openai-agents

!pip install sendgrid

In [42]:
from agents import Agent

from agents import trace

from agents import Runner

from agents import WebSearchTool

from agents import gen_trace_id

from agents import function_tool

from agents.model_settings import ModelSettings

from pydantic import BaseModel

import asyncio

import os

from IPython.display import display, Markdown

import sendgrid

from sendgrid import SendGridAPIClient

from sendgrid.helpers.mail import To

from sendgrid.helpers.mail import Email

from sendgrid.helpers.mail import Mail

from sendgrid.helpers.mail import Content

from typing import Dict

from google.colab import userdata

In [43]:
openai_api_key = os.environ["OPENAI_API_KEY"] = userdata.get("OpenAI_API_Key")

In [44]:
research_agent_instructions = """

You are a very sucessful and award-winning research assistant.

Given a search term, you will always search the web for that term and generate a robust summary of the results.

The summary must be strictly only 2 to 3 paragraphs and not more than 400 words.

Capture only the essential and feasible points, write it in a very easy to understand way yet in extremely impacting tone.

Do not include any unneccasary information other than the summary itself

"""

In [45]:
research_agent = Agent(

                       name = "Research Assistant",

                       instructions = research_agent_instructions,

                       tools = [WebSearchTool(search_context_size = "low")],

                       model_settings = ModelSettings(tool_choice = "required"),

                       model = "gpt-4o"
)

query = "AI's contribution in tackling and mitigating the impacts of Climate Change."

with trace("Web Search"):

  web_search_result = await Runner.run(research_agent, query)

print(display(Markdown(web_search_result.final_output)))

Artificial Intelligence (AI) is playing a pivotal role in combating climate change by enhancing our ability to monitor environmental changes, predict climate-related events, and implement effective mitigation strategies. AI-driven models analyze vast datasets from satellites and sensors to track deforestation, monitor ocean health, and forecast extreme weather events like hurricanes and floods. For instance, the European Flood Awareness System utilizes AI to provide accurate flood forecasts across Europe, enabling timely evacuations and resource mobilization to minimize damage and ensure public safety. ([holisticseo.digital](https://www.holisticseo.digital/ai/benefit/climate-change/?utm_source=openai))

In the energy sector, AI optimizes the integration of renewable sources such as wind and solar into power grids, improving efficiency and reducing reliance on fossil fuels. AI-powered smart grids balance energy supply and demand, reducing waste and enhancing grid stability. Additionally, AI aids in sustainable agriculture by analyzing data on crop yields, soil health, and weather patterns, enabling farmers to make informed decisions that lead to more efficient resource use and reduced emissions. ([appen.com](https://www.appen.com/blog/how-ai-technology-is-revolutionizing-climate-change-mitigation?utm_source=openai))

Furthermore, AI contributes to climate adaptation by developing early warning systems for natural disasters, allowing communities to prepare and respond effectively. AI models can simulate the impact of rising sea levels on coastal infrastructure, assisting governments and businesses in formulating adaptation strategies. However, while AI offers significant opportunities in climate action, it also presents challenges, including the environmental costs associated with energy-intensive data centers. Therefore, careful implementation and balancing of AI's benefits and risks are crucial for its success in addressing climate change. ([ft.com](https://www.ft.com/content/2385e6ab-26ed-4e98-b70c-1d968246ea69?utm_source=openai)) 

None


In [46]:
how_many_searches = 3

research_planner_agent_instructions = f"You are a fellow and a very helpful research assistant. Given a query, come up with a set of web searches to best answer the query.Output {how_many_searches} terms to query for"

In [47]:
class WebSearchItem(BaseModel):

  reason: str
  "Your reasoning and thought process of why this search is important to the query."

  query: str
  "The search term for to use for web searches"

In [48]:
class WebSearchResultsList(BaseModel):

  web_search_results : list[WebSearchItem]

  """List of relevant web links of the query search result."""


In [49]:
research_planner_agent = Agent(

                               name = "Research Planner",

                               instructions = research_planner_agent_instructions,

                               output_type = WebSearchResultsList,

                               model = "gpt-4o"
)

query = "AI's contribution in tackling and mitigating the impacts of Climate Change."

with trace("Web Search Results"):

   web_search_results_list = await Runner.run(research_planner_agent, query)

   for item in web_search_results_list.final_output.web_search_results:

       print(f"Reason: {item.reason}")

       print(f"Query: {item.query}")

       print("-" * 20) # Print a separator for clarity

Reason: To find information about how AI is being used specifically in climate modeling and prediction.
Query: AI in climate change modeling and prediction
--------------------
Reason: To explore AI-based solutions and innovations addressing climate change impacts.
Query: AI technologies for mitigating climate change
--------------------
Reason: To understand the role of AI in optimizing energy use and promoting renewable energy sources.
Query: AI in renewable energy and energy efficiency
--------------------


In [50]:
@function_tool

def send_email(subject: str, html_body: str) -> Dict[str, str]:

  """Send out a given email with the given subject and html body. """

  sendgrid_client = SendGridAPIClient(api_key = userdata.get("SENDGRID_API_KEY"))

  to_email = To("faiaza037@gmail.com")

  from_email = Email("faiazrex8@gmail.com")

  content = Content("text/html", html_body)

  mail = Mail(to_email, from_email, subject, content).get()

  response = sendgrid_client.client.send.mail.post(request_body = mail)

  return {"status" : "success"}

send_email

FunctionTool(name='send_email', description='Send out a given email with the given subject and html body.', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7d08b5e3b6a0>, strict_json_schema=True, is_enabled=True)

In [51]:
email_agent_instructions = """

You send nice HTML formatted emails based on a detailed report as you will be provided a detailed report.

Use your available tool to send one email providing the report is converted to a clean, well structured HTML with appropriate subject line.

"""

email_agent = Agent(

                    name = "Email Agent",

                    instructions = email_agent_instructions,

                    tools = [send_email],

                    model = "gpt-4o"

                    )

In [52]:
research_writer_instructions =  """

You are a well experienced research writer tasked to preparing a cohesive report for a research query.

You will be provided with the original query, and a few initial research draft done by a research assistant.

You must first come up with the outline of the report that talks about the structure and flow of the report.

Finally, generate the report and give that as your final output.

The final output should be in a markdown format, and it should be lengthy and detailed, yet easy to understand.

Deliver at least 5 to 7 pages of content of minimum 500 to 700 words.

"""

In [53]:
class ReportData(BaseModel):

  summary: str
  "A summary of the report generated"

  markdown_report: str
  "The final report"

  follow_up_questions: list[str]
  "Further relevant and associated research areas to discuss and explore further"

In [54]:
research_writer = Agent(

                        name = "Research Writer",

                        instructions = research_writer_instructions,

                        model = "gpt-4o",

                        output_type = ReportData
)

In [55]:
async def web_search_plan(query: str):

  """Use the research planner agent to plan which searches to run for the given query."""

  print("Planning web search for given query...")

  result = await Runner.run(research_planner_agent, f"Query: {query}")

  print(f"Will perform {len(result.final_output.web_search_results)} searches!")

  return result.final_output


async def perform_web_search(search_plan : WebSearchResultsList):

  """ Calling search() for each item in search results list."""

  print("Searching...")

  task = [asyncio.create_task(search(item)) for item in search_plan.web_search_results]

  result = await asyncio.gather(*task)

  print("Finished Searching!")

  return result


async def search(item: WebSearchItem):

  """ Using this search agent to run a web search to each item in the Web search plan/result list. """

  input = f"Query: {item.query}, Reason: {item.reason}"

  result = await Runner.run(research_agent, input)

  return result.final_output


async def write_research_report(query: str, search_results: list):

  """ Prepare a research report based on the web search results """

  print("Analyzing and preparing...")

  input = f"Initial Query: {query}, Search Results: {search_results}"

  result = await Runner.run(research_writer, input)

  print("Research report is ready!")

  return result.final_output


async def send_research_report_email(report: ReportData):

  print("Sending email with research report...")

  result = await Runner.run(email_agent, report.markdown_report)

  print("Email Sent Sucessfully with final draft of Research Report attached.")

  return result.final_output

In [56]:
query = "AI's contribution in tackling and mitigating the impacts of Climate Change."

with trace("Research Report Generation"):

  print("Initializing research....")

  web_search_plan_result = await web_search_plan(query)

  web_search_result = await perform_web_search(web_search_plan_result)

  research_report = await write_research_report(query, web_search_result)

  final_result = await send_research_report_email(research_report)

  print("Entire task completed sucessfully end to end! Yupieeeee!!!! :-D")

Initializing research....
Planning web search for given query...
Will perform 3 searches!
Searching...
Finished Searching!
Analyzing and preparing...
Research report is ready!
Sending email with research report...
Email Sent Sucessfully with final draft of Research Report attached.
Entire task completed sucessfully end to end! Yupieeeee!!!! :-D
